# Predictive Maintenance mit SCANIA-Daten – Data Preperation Testing

**Projekt:** Bachelorarbeit Data Science  
**Thema:** 
**Datengrundlage:** SCANIA Component X Dataset  
**Autor:** Justin Stange-Heiduk  
**Betreuung:** Dr. Martin Prause  
**Ziel:** Erstellen und testen der Daten Vorbereitung Funktionen  

---

**Erstellt:** 2025-08-05  
**Letzte Änderung:** 2025-07-25


---

### 1. Laden der Daten
### 2. Interpolation fehlender Werte
### 3. Differenzbildung bei kumulativen Sensorwerten
### 4. Rolling Windows
### 5. Feature-Extraktion mit tsfresh
### 6. Label-Erstellung
### 7. Feature-Auswahl
### 8. Resampling (Unter-/Oversampling) zur Klassenbalance

In [90]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction.feature_calculators import *
from tsfresh.feature_extraction.settings import from_columns
from tsfresh.utilities.distribution import MultiprocessingDistributor
from lifelines import CoxPHFitter
from lifelines.statistics import proportional_hazard_test
import gc
from typing import Dict
import json

In [2]:
%run CommonFunctions.ipynb

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

### raw_date laden

In [ ]:
raw_data = load_all_raw_data()

## Data Preperation for Survivial Analyse

### Interpolation missing Values

In [ ]:
for key in raw_data.keys():
    locals()[f"readouts_{key}"] = raw_data[key]["readouts"]

In [ ]:
def interpolate_readout_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Performs linear interpolation of missing values in time series sensor data per vehicle.

    The function:
    - Sorts the input DataFrame by 'vehicle_id' and 'time_step'
    - Applies linear interpolation separately for each vehicle
    - Interpolates all numeric feature columns except 'vehicle_id' and 'time_step'
    - Fills missing values in both directions (forward and backward)

    Args:
        df (pd.DataFrame): The raw readout DataFrame containing sensor values, 
            including the columns 'vehicle_id' and 'time_step'

    Returns:
        pd.DataFrame: A new DataFrame with interpolated sensor features. 
            Original index is not preserved. Interpolation is applied only to 
            feature columns; identifiers remain unchanged.
    """
    # Sort by vehicle_id and time_step
    df = df.sort_values(by=["vehicle_id", "time_step"])

    # Interpolate numeric columns, excluding 'vehicle_id' and 'time_step'
    feature_cols = df.select_dtypes(include=["number"]).columns.difference(["vehicle_id", "time_step"])
    df[feature_cols] = df.groupby("vehicle_id")[feature_cols].transform(lambda x: x.interpolate(method="linear", limit_direction="both"))

    # Reset index to avoid keeping the original index
    return df.reset_index(drop=True)

In [ ]:
for key in raw_data.keys():
    locals()[f"readouts_inter_{key}"] = interpolate_readout_df(raw_data[key]["readouts"])

In [ ]:
def interpolate_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Interpolate missing values in the DataFrame

    Args:
        df (pd.DataFrame): The input raw_data with potential missing values.

    """
    
    for key in raw_data.keys():
        locals()[f"readouts_inter_{key}"] = interpolate_readout_df(raw_data[key]["readouts"])
    


In [ ]:
feature_cols = readouts_train.select_dtypes(include=["number"]).columns.difference(["vehicle_id", "time_step"])

original_stats_train = readouts_train[feature_cols].describe()
interp_stats_train = readouts_inter_train[feature_cols].describe()

original_stats_test = readouts_test[feature_cols].describe()
interp_stats_test = readouts_inter_test[feature_cols].describe()

original_stats_validation = readouts_validation[feature_cols].describe()
interp_stats_validation = readouts_inter_validation[feature_cols].describe()

print("Stats (train):")
display(original_stats_train/interp_stats_train)
print("Stats (test):")
display(original_stats_test/interp_stats_test)
print("Stats (validation):")
display(original_stats_validation/interp_stats_validation)

In [ ]:
for split in raw_data.keys():
        df = raw_data[split]['readouts']
        locals()[f"{split}_missing_values"] = df.isnull().sum().sum()
        

readouts_inter_train_missing_values = readouts_inter_train.isnull().sum().sum()
readouts_inter_test_missing_values = readouts_inter_test.isnull().sum().sum()
readouts_inter_validation_missing_values = readouts_inter_validation.isnull().sum().sum()

print(f"Missing values in train: before={train_missing_values}, after={readouts_inter_train_missing_values}")
print(f"Missing values in test: before={test_missing_values}, after={readouts_inter_test_missing_values}")
print(f"Missing values in validation: before={validation_missing_values}, after={readouts_inter_validation_missing_values}")


In [ ]:
missing_columns = readouts_inter_train.loc[:, readouts_inter_train.isna().any()].columns.tolist()

display(readouts_inter_train[readouts_inter_train[missing_columns].isna().any(axis=1)])

In [ ]:
missing_columns = readouts_inter_test.loc[:, readouts_inter_test.isna().any()].columns.tolist()
display(readouts_inter_test[readouts_inter_test[missing_columns].isna().any(axis=1)])

#### Remove vehicles with nan Value

In [ ]:
def remove_vehicles_with_nans_from_split(
    readouts_df: pd.DataFrame,
    raw_split_data: dict,
    split_name: str
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Removes vehicles with remaining NaN values from readouts, labels, and specs 
    for a given dataset split (train, test, validation).

    Args:
        readouts_df (pd.DataFrame): Interpolated readouts DataFrame.
        raw_split_data (dict): Dictionary containing 'tte' or 'labels' and 'spec' DataFrames.
        split_name (str): Name of the data split ("train", "test", or "validation").

    Returns:
        tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: Filtered readouts, labels/tte, and specs.
    """
    # NaN-Spalten identifizieren
    missing_cols = readouts_df.columns[readouts_df.isna().any()].tolist()

    if not missing_cols:
        print(f"[{split_name}] Keine fehlenden Werte – keine Fahrzeuge entfernt.")
        label_df = raw_split_data["tte"] if split_name == "train" else raw_split_data["labels"]
        return readouts_df.copy(), label_df.copy(), raw_split_data["spec"].copy()

    # Fahrzeuge mit NaNs finden
    vehicles_with_nans = (
        readouts_df.loc[readouts_df[missing_cols].isna().any(axis=1), "vehicle_id"]
        .unique()
    )

    print(f"[{split_name}] {len(vehicles_with_nans)} Fahrzeug(e) mit NaNs entfernt.")
    print(f"   Entfernte vehicle_ids: {list(vehicles_with_nans)}")

    # Entsprechende DataFrames bereinigen
    readouts_clean = readouts_df[~readouts_df["vehicle_id"].isin(vehicles_with_nans)].copy()

    label_key = "tte" if split_name == "train" else "labels"
    labels_clean = raw_split_data[label_key][
        ~raw_split_data[label_key]["vehicle_id"].isin(vehicles_with_nans)
    ].copy()

    specs_clean = raw_split_data["spec"][
        ~raw_split_data["spec"]["vehicle_id"].isin(vehicles_with_nans)
    ].copy()

    return readouts_clean, labels_clean, specs_clean


In [ ]:
readouts_cleaned_train, tte_train_df, specs_train_df = remove_vehicles_with_nans_from_split(
    readouts_inter_train, raw_data["train"], "train"
)

readouts_cleaned_test, tte_test_df, specs_test_df = remove_vehicles_with_nans_from_split(
    readouts_inter_test, raw_data["test"], "test"
)

readouts_cleaned_validation, tte_val_df, specs_val_df = remove_vehicles_with_nans_from_split(
    readouts_inter_validation, raw_data["validation"], "validation"
)


In [ ]:
readouts_inter_train_missing_values = readouts_cleaned_train.isnull().sum().sum()
readouts_inter_test_missing_values = readouts_cleaned_test.isnull().sum().sum()
readouts_inter_validation_missing_values = readouts_cleaned_validation.isnull().sum().sum()

print(f"Missing values in train: before={train_missing_values}, after={readouts_inter_train_missing_values}")
print(f"Missing values in test: before={test_missing_values}, after={readouts_inter_test_missing_values}")
print(f"Missing values in validation: before={validation_missing_values}, after={readouts_inter_validation_missing_values}")


### Differenzbildung bei kumulativen Sensorwerten

In [ ]:
def compute_differences_per_vehicle_test(readouts_df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes first-order differences for cumulative sensor features per vehicle.
    For the first time step, uses the original value instead of NaN or 0.
    Test with columns _diff for each sensor bin.

    Args:
        readouts_df (pd.DataFrame): Readouts DataFrame with 'vehicle_id' and 'time_step'.
        
    Returns:
        pd.DataFrame: DataFrame with additional '_diff' columns per selected sensor bin.
    """
    df = readouts_df.copy()
    
    
    bin_cols = [col for col in df.columns.difference(["vehicle_id", "time_step"])]

    for col in bin_cols:
        new_col = f"{col}_diff"

        # Berechne Differenz pro vehicle_id
        df[new_col] = df.groupby("vehicle_id")[col].diff()

        # Ersetze NaN im ersten Schritt durch den Originalwert
        first_values = df.groupby("vehicle_id")[col].transform("first")
        df[new_col] = df[new_col].fillna(first_values)

    return df


In [ ]:
readouts_diff_train = compute_differences_per_vehicle_test(readouts_cleaned_train)

In [ ]:
display(readouts_diff_train[readouts_diff_train["vehicle_id"] == 5][["427_0", "427_0_diff"]])

In [ ]:
def compute_differences_per_vehicle(readouts_df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes first-order differences for cumulative sensor features per vehicle.
    For the first time step, uses the original value instead of NaN or 0.
    

    Args:
        readouts_df (pd.DataFrame): Readouts DataFrame with 'vehicle_id' and 'time_step'.
        
    Returns:
        pd.DataFrame: DataFrame with additional '_diff' columns per selected sensor bin.
    """
    df = readouts_df.copy()
    
    
    bin_cols = [col for col in df.columns.difference(["vehicle_id", "time_step"])]

    for col in bin_cols:
        
                
        first_values = df.groupby("vehicle_id")[col].transform("first")
        # Berechne Differenz pro vehicle_id
        df[col] = df.groupby("vehicle_id")[col].diff()

        # Ersetze NaN im ersten Schritt durch den Originalwert
        df[col] = df[col].fillna(first_values)

    return df


In [ ]:
readouts_diff_train = compute_differences_per_vehicle(readouts_cleaned_train)
readouts_diff_test = compute_differences_per_vehicle(readouts_cleaned_test)
readouts_diff_validation = compute_differences_per_vehicle(readouts_cleaned_validation)

In [ ]:
save_df(readouts_diff_train, ordner="02_intermediate", name="readouts_diff_train")
save_df(readouts_diff_test, ordner="02_intermediate", name="readouts_diff_test")
save_df(readouts_diff_validation, ordner="02_intermediate", name="readouts_diff_validation")

In [6]:
readouts_diff_train = load_df(ordner="02_intermediate", name="readouts_diff_train")
# readouts_diff_test = load_df(ordner="02_intermediate", name="readouts_diff_test")
# readouts_diff_validation = load_df(ordner="02_intermediate", name="readouts_diff_validation")

In [26]:
display(readouts_diff_test.head())

,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,167_4,167_5,167_6,167_7,167_8,167_9,309_0,272_0,272_1,272_2,272_3,272_4,272_5,272_6,272_7,272_8,272_9,835_0,370_0,291_0,291_1,291_2,291_3,291_4,291_5,291_6,291_7,291_8,291_9,291_10,158_0,158_1,158_2,158_3,158_4,158_5,158_6,158_7,158_8,158_9,100_0,459_0,459_1,459_2,459_3,459_4,459_5,459_6,459_7,459_8,459_9,459_10,459_11,459_12,459_13,459_14,459_15,459_16,459_17,459_18,459_19,397_0,397_1,397_2,397_3,397_4,397_5,397_6,397_7,397_8,397_9,397_10,397_11,397_12,397_13,397_14,397_15,397_16,397_17,397_18,397_19,397_20,397_21,397_22,397_23,397_24,397_25,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
0,1,4.4,94410.0,7982.0,3115020.0,2625.0,1304.0,366624.0,232040.0,147104.0,422728.0,1651432.0,223425.0,55640.0,8816.0,3449.0,35.0,589556.0,165737.0,107063.0,255684.0,3008236.0,173097.0,6589.0,0.0,0.0,0.0,4466869.0,23994.0,328.0,184.0,237.0,329.0,104.0,48.0,84.0,76.0,60.0,144.0,28.0,132118.0,1565102.0,1976016.0,469225.0,314398.0,312200.0,194684.0,135352.0,150738.0,230483.0,232410.0,199.198833,54.364944,45.376778,51.823889,63.903611,77.826611,103.357333,122.073778,445.946667,548.500944,347.169167,372.139389,386.215167,499.586222,801.094944,3222.756944,2.499500,0.111111,0.019500,0.000000,263249.0,239652.0,280203.0,26432.0,21462.0,2674.0,409690.0,144340.0,519841.0,5432.0,1408.0,77.0,53152.0,67929.0,242384.0,5194.0,1653.0,63.0,8961.0,41455.0,127793.0,6974.0,952.0,14.0,5489.0,29540.0,72548.0,9696.0,1309.0,7.0,399.0,31843.0,89138.0,24101.0,5112.0,7.0
1,1,5.0,10995.0,1196.0,352220.0,600.0,1.0,24272.0,20488.0,10720.0,26761.0,205368.0,30168.0,12352.0,2232.0,0.0,25.0,33452.0,20141.0,11066.0,11605.0,353243.0,29744.0,0.0,0.0,0.0,0.0,476832.0,1899.0,14.0,12.0,14.0,16.0,9.0,0.0,16.0,16.0,2.0,18.0,2.0,25970.0,151032.0,151480.0,42910.0,41188.0,53354.0,34790.0,19334.0,22652.0,42112.0,16855.0,15.360611,3.551833,4.019444,4.591333,7.239278,8.834500,15.243556,22.044667,58.584056,60.476056,41.487333,45.604333,47.604278,81.691278,94.827500,457.780167,0.854333,0.045944,0.002111,0.005111,30016.0,25046.0,42129.0,4692.0,4866.0,791.0,27671.0,8115.0,28063.0,407.0,161.0,28.0,4040.0,6295.0,35646.0,672.0,281.0,7.0,752.0,4970.0,22695.0,932.0,161.0,0.0,688.0,4229.0,10599.0,2346.0,532.0,0.0,56.0,3906.0,17228.0,4067.0,308.0,0.0
2,1,8.2,109035.0,14170.0,3618550.0,4785.0,3592.0,70344.0,125736.0,155192.0,348912.0,1923624.0,545904.0,95384.0,9392.0,1568.0,20.0,143452.0,115918.0,114554.0,275550.0,3529889.0,351791.0,6083.0,0.0,0.0,0.0,4711729.0,45207.0,130.0,84.0,136.0,246.0,69.0,30.0,107.0,80.0,22.0,168.0,42.0,188454.0,1158864.0,1804796.0,569394.0,466060.0,363244.0,258580.0,204386.0,256116.0,504770.0,138354.0,104.317944,36.328889,38.761167,48.907000,63.916778,81.614556,123.079444,167.179889,688.146167,885.584722,561.994056,735.728611,678.722722,765.459167,1277.286444,3664.576000,4.009444,0.181833,0.012722,0.000000,300644.0,292754.0,419727.0,46873.0,42980.0,6671.0,188693.0,129928.0,283080.0,5454.0,1610.0,231.0,19047.0,90503.0,308315.0,7134.0,2122.0,98.0,6748.0,60984.0,168652.0,8828.0,1589.0,63.0,6545.0,41678.0,114943.0,14057.0,2675.0,14.0,665.0,58268.0,187096.0,57428.0,11235.0,7.0
3,1,9.8,45750.0,5772.0,1611060.0,4515.0,1616.0,32488.0,77088.0,50968.0,166512.0,772576.0,308544.0,81320.0,10064.0,6008.0,140.0,45056.0,88297.0,35684.0,148709.0,1520871.0,236071.0,15213.0,0.0,0.0,0.0,2169949.0,16569.0,101.0,76.0,96.0,120.0,36.0,40.0,46.0,28.0,35.0,114.0,12.0,87346.0,732466.0,796852.0,215503.0,179508.0,135394.0,103726.0,83594.0,114058.0,211106.0,82053.0,51.635111,23.030667,19.974167,24.711500,31.951611,48.177944,63.630444,85.909722,336.708444,496.825444,323.425111,375.903944,340.031000,308.924889,447.205389,1238.801778,1.587500,0.032944,0.026167,0.000000,221039.0,173313.0,196364.0,23870.0,20471.0,3136.0,43484.0,74487.0,87389.0,3354.0,1547.0,126.0,7077.0,44625.0,97034.0,3738.0,1521.0,84.0,3304.0,29254.0,67040.0,4483.0,771.0,14.0,2667.0,19204.0,46236.0,6609.0,1198.0,28.0,378.0,24332.0,84714.0,29534.0,7399.

In [ ]:
display(readouts_diff_train[readouts_diff_train["vehicle_id"] == 0]["158_1"])

### Rolling Windows (Index-based, z. B. w ∈ {8})

In [7]:
def create_all_fixed_time_index_windows_faster(
    readouts_df: pd.DataFrame,
    window_sizes: list[float]
) -> pd.DataFrame:
    base_cols = ["vehicle_id", "time_step"]
    sensor_cols = [c for c in readouts_df.columns if c not in base_cols]

    # Melt einmalig in long-Format
    melted = readouts_df.melt(
        id_vars=base_cols,
        value_vars=sensor_cols,
        var_name="kind",
        value_name="value"
    )

    # Optimierte dtypes
    melted = melted.astype({
        "vehicle_id": "uint16",          # reicht völlig für ≤ 65535 Fahrzeuge
        "time_step": "float64",
        "kind": "category",              # viele Wiederholungen → spart RAM
        "value": "float32",              # falls hohe Präzision nicht nötig
    }, copy=False)

    melted = melted.sort_values(["vehicle_id", "time_step"], kind="mergesort", ignore_index=True)
    print(f"anzhal an vehicle ids: {melted['vehicle_id'].nunique()}")

    # Gruppieren (RAM-schonend) – dict bleibt
    vehicle_groups = {vid: g for vid, g in melted.groupby("vehicle_id", sort=False)}
    n_vehicles = len(vehicle_groups)

    out_parts: list[pd.DataFrame] = []

    for w in window_sizes:
        print(f"Windows sized {w} wird gemacht")

        for vid, g in tqdm(
            vehicle_groups.items(),
            total=n_vehicles,
            desc=f"Vehicles (w={w})",
            leave=True
        ):
            t = g["time_step"].to_numpy()
            uniq_t = np.unique(t)
            if uniq_t.size == 0:
                continue

            starts = np.searchsorted(t, uniq_t - w, side="right")
            ends   = np.searchsorted(t, uniq_t,     side="left")

            def _fmt_ts(x: float) -> str:
                s = f"{x:.1f}"
                return s.rstrip("0").rstrip(".") if "." in s else str(x)

            # Optional: Nur bei Bedarf!
            id_suffixes = [f"w{w}__id{vid}_t{_fmt_ts(ct)}" for ct in uniq_t]

            for i_start, i_end, win_id, ct in zip(starts, ends, id_suffixes, uniq_t):
                if i_end <= i_start:
                    # Fallback: nimm nur den letzten Wert als 1-Zeilen-Window
                    fallback_time = t[i_end - 1]
                    fallback_rows = g[g["time_step"] == fallback_time].copy()
                    fallback_rows["id"] = win_id
                    fallback_rows["time_step_current"] = np.float64(ct)
                    out_parts.append(fallback_rows)
                    continue
                win_slice = g.iloc[i_start:i_end].copy()

                win_slice["id"] = win_id
                win_slice["time_step_current"] = np.float64(ct)

                out_parts.append(win_slice)

    if not out_parts:
        return pd.DataFrame(columns=["id", "vehicle_id", "time_step", "time_step_current", "kind", "value"])

    final = pd.concat(out_parts, ignore_index=True)

    final = final.astype({
        "vehicle_id": "uint16",
        "time_step": "float64",
        "time_step_current": "float64",
        "kind": "category",
        "value": "float32",
        "id": "object",  # hier keine Alternative, da unique string – ggf. später entfernen
    }, copy=False)


    print(f"anzhal an vehicle ids: {final['vehicle_id'].nunique()}")
    # Alle float32-Spalten auf 1 Nachkommastelle runden
    float_cols = final.select_dtypes(include=["float32"]).columns
    final[float_cols] = final[float_cols].round(1)


    return final[["id", "vehicle_id", "time_step", "time_step_current", "kind", "value"]]


In [ ]:
windowed_df = create_all_fixed_time_index_windows_faster(
    readouts_df=readouts_diff_train[readouts_diff_train["vehicle_id"] == 0],
    window_sizes=[8],
)

display(windowed_df.kind.unique().shape)
display(windowed_df[windowed_df["vehicle_id"] == 0][windowed_df["kind"] == "158_1"].head(50))
display(readouts_diff_train[readouts_diff_train["vehicle_id"] == 0][["time_step", "158_1"]].head(50))

In [8]:
combined_window_df_8_train = create_all_fixed_time_index_windows_faster(
    readouts_df=readouts_diff_train,
    window_sizes=[8]
)

anzhal an vehicle ids: 23548
Windows sized 8 wird gemacht


Vehicles (w=8):   0%|          | 0/23548 [00:00<?, ?it/s]

anzhal an vehicle ids: 23548


In [9]:
save_df(df = combined_window_df_8_train, ordner="03_primary", name = "combined_window_df_8_train")


In [40]:
combined_window_df_8_train = load_df(ordner="03_primary", name = "combined_window_df_8_train")

In [8]:
def last_readout_window_per_window(df: pd.DataFrame, window_size: int) -> pd.DataFrame:
    """
    Funktion, um das letzte Readout innerhalb eines bestimmten Zeitfensters für jedes Fahrzeug zu ermitteln.

    Args:
        df (pd.DataFrame): Das Eingabe-DataFrame mit Fahrzeugdaten.
        window_size (int): Die Größe des Zeitfensters.

    Returns:
        pd.DataFrame: Ein DataFrame mit den letzten Readouts innerhalb des Zeitfensters für jedes Fahrzeug.
    """

    # Schritt 1: Max time_step je Fahrzeug
    last_steps = (
        readouts_diff_test
        .groupby("vehicle_id", sort=False)["time_step"]
        .max()
        .reset_index()
        .rename(columns={"time_step": "last_time"})
    )

    # Schritt 2: Join + Filter auf Time-Window
    merged = readouts_diff_test.merge(last_steps, on="vehicle_id", how="left")

    # Schritt 3: Zeitbereich filtern (t - n)
    filtered = merged[
        (merged["time_step"] >= merged["last_time"] - window_size)
        & (merged["time_step"] <= merged["last_time"])
    ]

    # Schritt 4: Letztes Readout pro Fahrzeug und Zeitfenster
    return filtered.drop(columns=["last_time"])

In [86]:
last_readout_window_test = last_readout_window_per_window(readouts_diff_test, window_size=8)
last_readout_window_validation = last_readout_window_per_window(readouts_diff_validation, window_size=8)


In [58]:
display(last_readout_window_test[["vehicle_id", "time_step", "397_35"]][last_readout_window_test["vehicle_id"] == 9].sort_values(by="time_step"))

,vehicle_id,time_step,397_35
236,9,340.0,1471.0


In [87]:
combined_window_df_8_test = create_all_fixed_time_index_windows_faster(
    readouts_df=last_readout_window_test,
    window_sizes=[8]
)

combined_window_df_8_validation = create_all_fixed_time_index_windows_faster(
    readouts_df=last_readout_window_validation,
    window_sizes=[8]
)

anzhal an vehicle ids: 5043
Windows sized 8 wird gemacht


Vehicles (w=8):   0%|          | 0/5043 [00:00<?, ?it/s]

anzhal an vehicle ids: 5043
anzhal an vehicle ids: 5043
Windows sized 8 wird gemacht


Vehicles (w=8):   0%|          | 0/5043 [00:00<?, ?it/s]

anzhal an vehicle ids: 5043


In [5]:
def filter_latest_windows_per_vehicle_and_kind(df: pd.DataFrame) -> pd.DataFrame:
    """
    Behalte nur die Zeilen mit dem maximalen `time_step_current` pro vehicle_id und kind.
    Mehrere Zeilen pro Kombination sind möglich, wenn sie denselben `time_step_current` haben.

    Args:
        df (pd.DataFrame): Long-formatiertes Sliding Window DataFrame mit Spalten:
                           ['vehicle_id', 'kind', 'time_step_current', ...]

    Returns:
        pd.DataFrame: Gefiltertes DataFrame mit nur den letzten Windows pro vehicle_id und kind.
    """
    # Maximalen time_step_current pro vehicle_id + kind bestimmen
    max_time = (
        df.groupby(["vehicle_id", "kind"], sort=False)["time_step_current"]
        .transform("max")
    )

    # Nur Zeilen behalten, bei denen time_step_current == max
    return df[df["time_step_current"] == max_time].copy()


In [91]:
combined_window_df_8_test = filter_latest_windows_per_vehicle_and_kind(combined_window_df_8_test)
combined_window_df_8_validation = filter_latest_windows_per_vehicle_and_kind(combined_window_df_8_validation)

/tmp/ipykernel_10489/1711113000.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["vehicle_id", "kind"], sort=False)["time_step_current"]


In [92]:
save_df(df = combined_window_df_8_test, ordner="03_primary", name = "combined_window_df_8_test")
save_df(df = combined_window_df_8_validation, ordner="03_primary", name = "combined_window_df_8_validation")


In [24]:
combined_window_df_8_test = load_df(ordner="03_primary", name = "combined_window_df_8_test")
combined_window_df_8_validation = load_df(ordner="03_primary", name = "combined_window_df_8_validation")

In [93]:
display(combined_window_df_8_test[combined_window_df_8_test["vehicle_id"] == 1][combined_window_df_8_test["kind"] == "171_0"].sort_values(by="time_step"))

/tmp/ipykernel_10489/4235577928.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(combined_window_df_8_test[combined_window_df_8_test["vehicle_id"] == 1][combined_window_df_8_test["kind"] == "171_0"].sort_values(by="time_step"))


,id,vehicle_id,time_step,time_step_current,kind,value
210,w8__id1_t224,1,216.8,224.0,171_0,118830.0
315,w8__id1_t224,1,222.6,224.0,171_0,149970.0


### tsfresh feature extraction

In [28]:
def extract_tsfresh_features_for_all_windows(
    df_windows: pd.DataFrame,
    anzahl_workers: int = 8
) -> pd.DataFrame:
    """
    Extrahiert die wichtigsten tsfresh-Features pro Sensor und Window-ID 
    (z. B. w8__id42_t123.4) im Long-Format.
    
    Args:
        df_windows (pd.DataFrame): DataFrame mit den Spalten:
            ['id', 'vehicle_id', 'time_step', 'time_step_current', 'kind', 'value']
        anzahl_workers (int): Anzahl paralleler Prozesse für tsfresh.

    Returns:
        pd.DataFrame: Wide-Format mit reduzierter Featureanzahl + vehicle_id + time_step_current.
    """

    distributor = MultiprocessingDistributor(
        n_workers=anzahl_workers,
        disable_progressbar=False,
        progressbar_title="tsfresh Feature Extraction"
    )

    selected_fc_parameters = {
        "mean": None,
        "median": None,
        "standard_deviation": None,
        "minimum": None,
        "maximum": None
    }

    features_df = extract_features(
        df_windows,
        column_id="id",
        column_sort="time_step",
        column_kind="kind",
        column_value="value",
        default_fc_parameters=selected_fc_parameters,
        impute_function=impute,
        distributor=distributor
    )

    # vehicle_id und time_step_current ergänzen
    id_metadata = df_windows[["id", "vehicle_id", "time_step_current"]].drop_duplicates("id")
    features_df = features_df.merge(id_metadata, how="left", left_index=True, right_on="id")
    features_df = features_df.set_index("id")
    features_df = features_df.rename(columns={"time_step_current": "time_step"})

    float_cols = features_df.select_dtypes(include=["float64"]).columns.difference(["vehicle_id", "time_step"])
    features_df[float_cols] = features_df[float_cols].astype("float32")

    return features_df

#### Extra funktions for chunk execution

In [4]:
def split_and_save_dataframe_by_id(
    df: pd.DataFrame,
    id_column: str,
    output_dir: Union[str, Path],
    n_parts: int = 10,
    file_prefix: str = "chunk"
) -> None:
    """
    Splits a DataFrame into `n_parts` based on unique values in the `id_column`
    and saves each chunk as a separate Parquet file.

    Args:
        df (pd.DataFrame): Input DataFrame in long format.
        id_column (str): Column name used to split the data (e.g., "id").
        output_dir (Union[str, Path]): Directory to save the Parquet chunks.
        n_parts (int, optional): Number of parts to split into. Defaults to 10.
        file_prefix (str, optional): Prefix for output files. Defaults to "chunk".

    Returns:
        None
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    unique_ids = df[id_column].unique()
    total_ids = len(unique_ids)
    chunk_size = total_ids // n_parts

    for i in range(n_parts):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < n_parts - 1 else total_ids
        ids_chunk = unique_ids[start:end]

        df_chunk = df[df[id_column].isin(ids_chunk)].copy()
        output_file = output_dir / f"{file_prefix}_{i+1:02}.parquet"

        df_chunk.to_parquet(output_file, index=False)
        print(f"[{i+1}/{n_parts}] Saved {len(df_chunk)} rows to {output_file}")


In [41]:
split_and_save_dataframe_by_id(
    df=combined_window_df_8_train,
    id_column="id",
    output_dir="../data/03_primary/train_chunks_parquet_pre_tsfresh",
    n_parts=10,
    file_prefix="train"
)

[1/10] Saved 27306090 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_01.parquet
[2/10] Saved 21131145 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_02.parquet
[3/10] Saved 20701485 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_03.parquet
[4/10] Saved 21361095 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_04.parquet
[5/10] Saved 20946345 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_05.parquet
[6/10] Saved 20242635 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_06.parquet
[7/10] Saved 20100780 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_07.parquet
[8/10] Saved 19699995 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_08.parquet
[9/10] Saved 20085660 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_09.parquet
[10/10] Saved 22785840 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_10.parquet


In [5]:
def extract_tsfresh_features_from_parquet_chunk(
    input_parquet: Union[str, Path],
    output_parquet: Union[str, Path],
    anzahl_workers: int = 8,
) -> None:
    """
    Loads a long-format window DataFrame from a Parquet file, performs tsfresh feature extraction,
    and writes the resulting wide-format features back to another Parquet file.

    Args:
        input_parquet (Union[str, Path]): Path to input parquet file (long format).
        output_parquet (Union[str, Path]): Path where to save extracted features (wide format).
        anzahl_workers (int, optional): Number of parallel processes for tsfresh. Default is 8.

    Returns:
        None
    """
    input_parquet = Path(input_parquet)
    output_parquet = Path(output_parquet)
    
    print(f"Loading: {input_parquet}")
    df = pd.read_parquet(input_parquet)


        
    distributor = MultiprocessingDistributor(
    n_workers=anzahl_workers,
    disable_progressbar=False,
    progressbar_title="tsfresh Feature Extraction"
        )
    
    selected_fc_parameters = {
        "mean": None,
        "median": None,
        "standard_deviation": None,
        "minimum": None,
        "maximum": None
    }

    features_df = extract_features(
        df,
        column_id="id",
        column_sort="time_step",
        column_kind="kind",
        column_value="value",
        default_fc_parameters=selected_fc_parameters,  # <<< geändert!
        impute_function=impute,
        distributor=distributor
    )

    id_metadata = df[["id", "vehicle_id", "time_step_current"]].drop_duplicates("id")
    features_df = features_df.merge(id_metadata, how="left", left_index=True, right_on="id")
    features_df = features_df.set_index("id")
    features_df = features_df.rename(columns={"time_step_current": "time_step"})

    float_cols = features_df.select_dtypes(include="float64").columns.difference(["vehicle_id", "time_step"])
    features_df[float_cols] = features_df[float_cols].astype("float32")

    print(f"💾 Saving: {output_parquet}")
    features_df.to_parquet(output_parquet)


In [4]:
gc.collect()

122

In [6]:

input_dir = Path("../data/03_primary/train_chunks_parquet_pre_tsfresh")
output_dir = Path("../data/03_primary/train_features_parquet")
output_dir.mkdir(exist_ok=True)

for f in sorted(input_dir.glob("*.parquet")):

    
    extract_tsfresh_features_from_parquet_chunk(
        input_parquet=f,
        output_parquet=output_dir / f"{f.stem}_features.parquet",
        anzahl_workers=5
    )
    gc.collect()  # Manuelles Aufräumen, um RAM zu schonen

Loading: ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_07.parquet


tsfresh Feature Extraction: 100%|██████████| 25/25 [29:33<00:00, 70.95s/it]  


💾 Saving: ../data/03_primary/train_features_parquet/train_07_features.parquet
Loading: ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_08.parquet


tsfresh Feature Extraction: 100%|██████████| 25/25 [29:40<00:00, 71.21s/it]  


💾 Saving: ../data/03_primary/train_features_parquet/train_08_features.parquet
Loading: ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_09.parquet


tsfresh Feature Extraction: 100%|██████████| 25/25 [29:48<00:00, 71.55s/it]  


💾 Saving: ../data/03_primary/train_features_parquet/train_09_features.parquet
Loading: ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_10.parquet


tsfresh Feature Extraction: 100%|██████████| 25/25 [29:51<00:00, 71.67s/it]  


💾 Saving: ../data/03_primary/train_features_parquet/train_10_features.parquet


In [29]:
readout_tsfresh_features_test = extract_tsfresh_features_for_all_windows(combined_window_df_8_test, anzahl_workers=6)


tsfresh Feature Extraction: 100%|██████████| 30/30 [01:23<00:00,  2.77s/it]


In [30]:
readout_tsfresh_features_validation = extract_tsfresh_features_for_all_windows(combined_window_df_8_validation, anzahl_workers=6)


tsfresh Feature Extraction: 100%|██████████| 30/30 [01:24<00:00,  2.81s/it]


#### Überprüfen ob die Feature Spalten auf float32 statt float64

In [ ]:
def check_float32_loss(series: pd.Series, tolerance: float = 1e-6) -> bool:
    """
    Prüft, ob eine float64-Spalte verlustfrei (bzw. toleranzbasiert) in float32 konvertiert werden kann.

    Args:
        series (pd.Series): Die Spalte, die geprüft werden soll.
        tolerance (float): Erlaubte maximale Abweichung.

    Returns:
        bool: True, wenn float32 möglich ist, False sonst.
    """
    series_float32 = series.astype("float32")
    max_diff = (series - series_float32.astype("float64")).abs().max()
    return max_diff < tolerance

In [ ]:
for col in readout_tsfresh_features_test.select_dtypes(include="float64"):
    if check_float32_loss(readout_tsfresh_features_test[col]):
        continue
    else:
        print(f"{col} lieber float64 behalten.")


### Überprüfen der tsfresh Daten

In [ ]:
display(readout_tsfresh_features_test.head(5))

In [33]:
display(combined_window_df_8_test[combined_window_df_8_test["vehicle_id"] == 1000][combined_window_df_8_test["kind"] == "100_0"].sort_values(by="time_step"))

/tmp/ipykernel_1256/2256109858.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(combined_window_df_8_test[combined_window_df_8_test["vehicle_id"] == 1000][combined_window_df_8_test["kind"] == "100_0"].sort_values(by="time_step"))


,id,vehicle_id,time_step,time_step_current,kind,value
27663,w8__id1000_t404.8,1000,398.2,404.8,100_0,68580.0
27768,w8__id1000_t404.8,1000,398.4,404.8,100_0,672.0
27873,w8__id1000_t404.8,1000,403.2,404.8,100_0,192006.0


In [34]:
display(readout_tsfresh_features_test.head(10))

,100_0__mean,100_0__median,100_0__standard_deviation,100_0__minimum,100_0__maximum,158_0__mean,158_0__median,158_0__standard_deviation,158_0__minimum,158_0__maximum,158_1__mean,158_1__median,158_1__standard_deviation,158_1__minimum,158_1__maximum,158_2__mean,158_2__median,158_2__standard_deviation,158_2__minimum,158_2__maximum,158_3__mean,158_3__median,158_3__standard_deviation,158_3__minimum,158_3__maximum,158_4__mean,158_4__median,158_4__standard_deviation,158_4__minimum,158_4__maximum,158_5__mean,158_5__median,158_5__standard_deviation,158_5__minimum,158_5__maximum,158_6__mean,158_6__median,158_6__standard_deviation,158_6__minimum,158_6__maximum,158_7__mean,158_7__median,158_7__standard_deviation,158_7__minimum,158_7__maximum,158_8__mean,158_8__median,158_8__standard_deviation,158_8__minimum,158_8__maximum,158_9__mean,158_9__median,158_9__standard_deviation,158_9__minimum,158_9__maximum,167_0__mean,167_0__median,167_0__standard_deviation,167_0__minimum,167_0__maximum,167_1__mean,167_1__median,167_1__standard_deviation,167_1__minimum,167_1__maximum,167_2__mean,167_2__median,167_2__standard_deviation,167_2__minimum,167_2__maximum,167_3__mean,167_3__median,167_3__standard_deviation,167_3__minimum,167_3__maximum,167_4__mean,167_4__median,167_4__standard_deviation,167_4__minimum,167_4__maximum,167_5__mean,167_5__median,167_5__standard_deviation,167_5__minimum,167_5__maximum,167_6__mean,167_6__median,167_6__standard_deviation,167_6__minimum,167_6__maximum,167_7__mean,167_7__median,167_7__standard_deviation,167_7__minimum,167_7__maximum,167_8__mean,167_8__median,167_8__standard_deviation,167_8__minimum,167_8__maximum,167_9__mean,167_9__median,167_9__standard_deviation,167_9__minimum,167_9__maximum,171_0__mean,171_0__median,171_0__standard_deviation,171_0__minimum,171_0__maximum,272_0__mean,272_0__median,272_0__standard_deviation,272_0__minimum,272_0__maximum,272_1__mean,272_1__median,272_1__standard_deviation,272_1__minimum,272_1__maximum,272_2__mean,272_2__median,272_2__standard_deviation,272_2__minimum,272_2__maximum,272_3__mean,272_3__median,272_3__standard_deviation,272_3__minimum,272_3__maximum,272_4__mean,272_4__median,272_4__standard_deviation,272_4__minimum,272_4__maximum,272_5__mean,272_5__median,272_5__standard_deviation,272_5__minimum,272_5__maximum,272_6__mean,272_6__median,272_6__standard_deviation,272_6__minimum,272_6__maximum,272_7__mean,272_7__median,272_7__standard_deviation,272_7__minimum,272_7__maximum,272_8__mean,272_8__median,272_8__standard_deviation,272_8__minimum,272_8__maximum,272_9__mean,272_9__median,272_9__standard_deviation,272_9__minimum,272_9__maximum,291_0__mean,291_0__median,291_0__standard_deviation,291_0__minimum,291_0__maximum,291_1__mean,291_1__median,291_1__standard_deviation,291_1__minimum,291_1__maximum,291_10__mean,291_10__median,291_10__standard_deviation,291_10__minimum,291_10__maximum,291_2__mean,291_2__median,291_2__standard_deviation,291_2__minimum,291_2__maximum,291_3__mean,291_3__median,291_3__standard_deviation,291_3__minimum,291_3__maximum,291_4__mean,291_4__median,291_4__standard_deviation,291_4__minimum,291_4__maximum,291_5__mean,291_5__median,291_5__standard_deviation,291_5__minimum,291_5__maximum,291_6__mean,291_6__median,291_6__standard_deviation,291_6__minimum,291_6__maximum,291_7__mean,291_7__median,291_7__standard_deviation,291_7__minimum,291_7__maximum,291_8__mean,291_8__median,291_8__standard_deviation,291_8__minimum,291_8__maximum,291_9__mean,291_9__median,291_9__standard_deviation,291_9__minimum,291_9__maximum,309_0__mean,309_0__median,309_0__standard_deviation,309_0__minimum,309_0__maximum,370_0__mean,370_0__median,370_0__standard_deviation,370_0__minimum,370_0__maximum,397_0__mean,397_0__median,397_0__standard_deviation,397_0__minimum,397_0__maximum,397_1__mean,397_1__median,397_1__standard_deviation,397_1__minimum,397_1__maximum,397_10__mean,397_10__median,397_10__standard_deviation,397_10__minimum,397_10__maximum,397_11__mean,397_11__median,397_11__standard_deviatio

#### Speichern der Tsfresh Daten

In [ ]:
def consolidate_parquet_files():
    """Consolidates multiple parquet files into a single parquet file."""
    input_dir = Path("../data/03_primary/train_features_parquet")
    output_file = Path("../data/03_primary/readout_tsfresh_features_train.parquet")

    dfs = [pd.read_parquet(f) for f in sorted(input_dir.glob("*.parquet"))]
    df_all = pd.concat(dfs, ignore_index=True)

    df_all.to_parquet(output_file, engine="pyarrow", index=False)
    print(f"Gespeichert unter: {output_file}")


In [ ]:
consolidate_parquet_files()

In [31]:
save_df(df = readout_tsfresh_features_test, ordner="03_primary", name = "readout_tsfresh_features_test")

In [32]:
save_df(df = readout_tsfresh_features_validation, ordner="03_primary", name = "readout_tsfresh_features_validation")

## Label Erstellung

### Transformation für train labels

In [16]:
label_train = load_specific_raw_data("train_tte")

In [17]:
display(label_train.head())

,vehicle_id,length_of_study_time_step,in_study_repair
0,0,510.0,0
1,2,281.8,0
2,3,293.4,0
3,4,210.0,0
4,5,360.4,0


In [5]:
def prepare_train_labels(df_labels: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare the training labels for survival analysis models.

    Args:
        df_labels (pd.DataFrame): DataFrame with columns ['vehicle_id', 'length_of_study_time_step', 'in_study_repair'].
                                  'in_study_repair' = 1 (event occurred), 0 (censored).

    Returns:
        pd.DataFrame: DataFrame with additional columns for Cox/RSF and AFT.
    """

    # Einheitliche Benennung für Event und Dauer
    df_labels = df_labels.copy()
    df_labels["duration"] = df_labels["length_of_study_time_step"]
    df_labels["event"] = df_labels["in_study_repair"]

    # AFT-Intervalle
    df_labels["upper_bound"] = np.where(df_labels["event"] == 1,
                                        df_labels["duration"],
                                        np.inf)
    return df_labels.drop(columns=["length_of_study_time_step", "in_study_repair"])

In [30]:
label_train_prepare = prepare_train_labels(label_train)

In [31]:
display(label_train_prepare[label_train_prepare["event"]==1].head())

,vehicle_id,duration,event,upper_bound
10,22,190.8,1,190.8
13,27,354.6,1,354.6
15,29,425.2,1,425.2
34,52,235.0,1,235.0
40,61,293.0,1,293.0


### Transformation für test und validation labels

In [32]:
label_test = load_specific_raw_data("test_labels")
label_validation = load_specific_raw_data("validation_labels")

In [33]:
display(label_test.head())

,vehicle_id,class_label
0,1,0
1,6,0
2,7,0
3,9,0
4,11,0


In [34]:
features_test = load_df(ordner="03_primary", name = "readout_tsfresh_features_test")

In [ ]:
##### Tests

In [42]:
subset = features_test[features_test["vehicle_id"].isin([1, 6, 7, 9, 11])]
display(subset)


,100_0__mean,100_0__median,100_0__standard_deviation,100_0__minimum,100_0__maximum,158_0__mean,158_0__median,158_0__standard_deviation,158_0__minimum,158_0__maximum,158_1__mean,158_1__median,158_1__standard_deviation,158_1__minimum,158_1__maximum,158_2__mean,158_2__median,158_2__standard_deviation,158_2__minimum,158_2__maximum,158_3__mean,158_3__median,158_3__standard_deviation,158_3__minimum,158_3__maximum,158_4__mean,158_4__median,158_4__standard_deviation,158_4__minimum,158_4__maximum,158_5__mean,158_5__median,158_5__standard_deviation,158_5__minimum,158_5__maximum,158_6__mean,158_6__median,158_6__standard_deviation,158_6__minimum,158_6__maximum,158_7__mean,158_7__median,158_7__standard_deviation,158_7__minimum,158_7__maximum,158_8__mean,158_8__median,158_8__standard_deviation,158_8__minimum,158_8__maximum,158_9__mean,158_9__median,158_9__standard_deviation,158_9__minimum,158_9__maximum,167_0__mean,167_0__median,167_0__standard_deviation,167_0__minimum,167_0__maximum,167_1__mean,167_1__median,167_1__standard_deviation,167_1__minimum,167_1__maximum,167_2__mean,167_2__median,167_2__standard_deviation,167_2__minimum,167_2__maximum,167_3__mean,167_3__median,167_3__standard_deviation,167_3__minimum,167_3__maximum,167_4__mean,167_4__median,167_4__standard_deviation,167_4__minimum,167_4__maximum,167_5__mean,167_5__median,167_5__standard_deviation,167_5__minimum,167_5__maximum,167_6__mean,167_6__median,167_6__standard_deviation,167_6__minimum,167_6__maximum,167_7__mean,167_7__median,167_7__standard_deviation,167_7__minimum,167_7__maximum,167_8__mean,167_8__median,167_8__standard_deviation,167_8__minimum,167_8__maximum,167_9__mean,167_9__median,167_9__standard_deviation,167_9__minimum,167_9__maximum,171_0__mean,171_0__median,171_0__standard_deviation,171_0__minimum,171_0__maximum,272_0__mean,272_0__median,272_0__standard_deviation,272_0__minimum,272_0__maximum,272_1__mean,272_1__median,272_1__standard_deviation,272_1__minimum,272_1__maximum,272_2__mean,272_2__median,272_2__standard_deviation,272_2__minimum,272_2__maximum,272_3__mean,272_3__median,272_3__standard_deviation,272_3__minimum,272_3__maximum,272_4__mean,272_4__median,272_4__standard_deviation,272_4__minimum,272_4__maximum,272_5__mean,272_5__median,272_5__standard_deviation,272_5__minimum,272_5__maximum,272_6__mean,272_6__median,272_6__standard_deviation,272_6__minimum,272_6__maximum,272_7__mean,272_7__median,272_7__standard_deviation,272_7__minimum,272_7__maximum,272_8__mean,272_8__median,272_8__standard_deviation,272_8__minimum,272_8__maximum,272_9__mean,272_9__median,272_9__standard_deviation,272_9__minimum,272_9__maximum,291_0__mean,291_0__median,291_0__standard_deviation,291_0__minimum,291_0__maximum,291_1__mean,291_1__median,291_1__standard_deviation,291_1__minimum,291_1__maximum,291_10__mean,291_10__median,291_10__standard_deviation,291_10__minimum,291_10__maximum,291_2__mean,291_2__median,291_2__standard_deviation,291_2__minimum,291_2__maximum,291_3__mean,291_3__median,291_3__standard_deviation,291_3__minimum,291_3__maximum,291_4__mean,291_4__median,291_4__standard_deviation,291_4__minimum,291_4__maximum,291_5__mean,291_5__median,291_5__standard_deviation,291_5__minimum,291_5__maximum,291_6__mean,291_6__median,291_6__standard_deviation,291_6__minimum,291_6__maximum,291_7__mean,291_7__median,291_7__standard_deviation,291_7__minimum,291_7__maximum,291_8__mean,291_8__median,291_8__standard_deviation,291_8__minimum,291_8__maximum,291_9__mean,291_9__median,291_9__standard_deviation,291_9__minimum,291_9__maximum,309_0__mean,309_0__median,309_0__standard_deviation,309_0__minimum,309_0__maximum,370_0__mean,370_0__median,370_0__standard_deviation,370_0__minimum,370_0__maximum,397_0__mean,397_0__median,397_0__standard_deviation,397_0__minimum,397_0__maximum,397_1__mean,397_1__median,397_1__standard_deviation,397_1__minimum,397_1__maximum,397_10__mean,397_10__median,397_10__standard_deviation,397_10__minimum,397_10__maximum,397_11__mean,397_11__median,397_11__standard_deviatio

In [4]:
label_test_timestep = label_test.merge(features_test[["vehicle_id", "time_step"]], on="vehicle_id", how="left")

NameError: name 'label_test' is not defined

In [39]:
display(label_test_timestep.head())

,vehicle_id,class_label,time_step
0,1,0,224.0
1,6,0,90.2
2,7,0,424.0
3,9,0,340.0
4,11,0,433.8


In [4]:
def prepare_test_validaten_labels(df_labels: pd.DataFrame, features_df: pd.DataFrame) -> pd.DataFrame:
    """
    Konvertiert Klassenlabels (0–4) in duration & event für Survival-Modelle.

    Args:
        test_labels_df (pd.DataFrame): Muss Spalten ['vehicle_id', 'time_step', 'class_label'] enthalten.

    Returns:
        pd.DataFrame mit Spalten ['vehicle_id', 'duration', 'event']
    """

    label_test_timestep = df_labels.merge(features_df[["vehicle_id", "time_step"]], on="vehicle_id", how="left")
    # Mittlere Restlebensdauer in Zeitschritten pro Klasse
    class_to_remaining_life = {
        4: 3,    # Klasse 4: innerhalb der nächsten 6 Zeitschritte
        3: 9,    # 6–12
        2: 18,   # 12–24
        1: 36,   # 24–48
        0: 48    # >48
    }

    df = label_test_timestep.copy()
    df["duration"] = df["class_label"].map(class_to_remaining_life)

    # Neue Duration = ab aktuellem time_step
    df["duration"] = df["time_step"] + df["duration"]

    # Event-Status: 1 = Defekt, 0 = Zensiert
    df["event"] = np.where(df["class_label"] != 0, 1, 0)

    df["upper_bound"] = np.where(df["event"] == 1,
                                        df["duration"],
                                        np.inf)

    return df.drop(columns=["time_step"])


In [51]:
labels_test_prepared = prepare_test_validaten_labels(label_test, features_test)

##### Tests

In [61]:
display(labels_test_prepared[labels_test_prepared["vehicle_id"].isin([1011,1430,1502,2084,1])].head(10))

,vehicle_id,class_label,duration,event,upper_bound
0,1,0,272.0,0,inf
151,1011,3,274.2,1,274.2
203,1430,1,187.6,1,187.6
215,1502,4,391.8,1,391.8
314,2084,2,172.6,1,172.6


In [62]:
subset = features_test[features_test["vehicle_id"].isin([1011,1430,1502,2084,1])]
display(subset)

,100_0__mean,100_0__median,100_0__standard_deviation,100_0__minimum,100_0__maximum,158_0__mean,158_0__median,158_0__standard_deviation,158_0__minimum,158_0__maximum,158_1__mean,158_1__median,158_1__standard_deviation,158_1__minimum,158_1__maximum,158_2__mean,158_2__median,158_2__standard_deviation,158_2__minimum,158_2__maximum,158_3__mean,158_3__median,158_3__standard_deviation,158_3__minimum,158_3__maximum,158_4__mean,158_4__median,158_4__standard_deviation,158_4__minimum,158_4__maximum,158_5__mean,158_5__median,158_5__standard_deviation,158_5__minimum,158_5__maximum,158_6__mean,158_6__median,158_6__standard_deviation,158_6__minimum,158_6__maximum,158_7__mean,158_7__median,158_7__standard_deviation,158_7__minimum,158_7__maximum,158_8__mean,158_8__median,158_8__standard_deviation,158_8__minimum,158_8__maximum,158_9__mean,158_9__median,158_9__standard_deviation,158_9__minimum,158_9__maximum,167_0__mean,167_0__median,167_0__standard_deviation,167_0__minimum,167_0__maximum,167_1__mean,167_1__median,167_1__standard_deviation,167_1__minimum,167_1__maximum,167_2__mean,167_2__median,167_2__standard_deviation,167_2__minimum,167_2__maximum,167_3__mean,167_3__median,167_3__standard_deviation,167_3__minimum,167_3__maximum,167_4__mean,167_4__median,167_4__standard_deviation,167_4__minimum,167_4__maximum,167_5__mean,167_5__median,167_5__standard_deviation,167_5__minimum,167_5__maximum,167_6__mean,167_6__median,167_6__standard_deviation,167_6__minimum,167_6__maximum,167_7__mean,167_7__median,167_7__standard_deviation,167_7__minimum,167_7__maximum,167_8__mean,167_8__median,167_8__standard_deviation,167_8__minimum,167_8__maximum,167_9__mean,167_9__median,167_9__standard_deviation,167_9__minimum,167_9__maximum,171_0__mean,171_0__median,171_0__standard_deviation,171_0__minimum,171_0__maximum,272_0__mean,272_0__median,272_0__standard_deviation,272_0__minimum,272_0__maximum,272_1__mean,272_1__median,272_1__standard_deviation,272_1__minimum,272_1__maximum,272_2__mean,272_2__median,272_2__standard_deviation,272_2__minimum,272_2__maximum,272_3__mean,272_3__median,272_3__standard_deviation,272_3__minimum,272_3__maximum,272_4__mean,272_4__median,272_4__standard_deviation,272_4__minimum,272_4__maximum,272_5__mean,272_5__median,272_5__standard_deviation,272_5__minimum,272_5__maximum,272_6__mean,272_6__median,272_6__standard_deviation,272_6__minimum,272_6__maximum,272_7__mean,272_7__median,272_7__standard_deviation,272_7__minimum,272_7__maximum,272_8__mean,272_8__median,272_8__standard_deviation,272_8__minimum,272_8__maximum,272_9__mean,272_9__median,272_9__standard_deviation,272_9__minimum,272_9__maximum,291_0__mean,291_0__median,291_0__standard_deviation,291_0__minimum,291_0__maximum,291_1__mean,291_1__median,291_1__standard_deviation,291_1__minimum,291_1__maximum,291_10__mean,291_10__median,291_10__standard_deviation,291_10__minimum,291_10__maximum,291_2__mean,291_2__median,291_2__standard_deviation,291_2__minimum,291_2__maximum,291_3__mean,291_3__median,291_3__standard_deviation,291_3__minimum,291_3__maximum,291_4__mean,291_4__median,291_4__standard_deviation,291_4__minimum,291_4__maximum,291_5__mean,291_5__median,291_5__standard_deviation,291_5__minimum,291_5__maximum,291_6__mean,291_6__median,291_6__standard_deviation,291_6__minimum,291_6__maximum,291_7__mean,291_7__median,291_7__standard_deviation,291_7__minimum,291_7__maximum,291_8__mean,291_8__median,291_8__standard_deviation,291_8__minimum,291_8__maximum,291_9__mean,291_9__median,291_9__standard_deviation,291_9__minimum,291_9__maximum,309_0__mean,309_0__median,309_0__standard_deviation,309_0__minimum,309_0__maximum,370_0__mean,370_0__median,370_0__standard_deviation,370_0__minimum,370_0__maximum,397_0__mean,397_0__median,397_0__standard_deviation,397_0__minimum,397_0__maximum,397_1__mean,397_1__median,397_1__standard_deviation,397_1__minimum,397_1__maximum,397_10__mean,397_10__median,397_10__standard_deviation,397_10__minimum,397_10__maximum,397_11__mean,397_11__median,397_11__standard_deviatio

### Ausführen der label transformations

In [8]:
labels_train_prepare = prepare_train_labels(load_specific_raw_data("train_tte"))
labels_test_prepared = prepare_test_validaten_labels(load_specific_raw_data("test_labels"), load_df(ordner="03_primary", name = "readout_tsfresh_features_test"))
labels_validation_prepared = prepare_test_validaten_labels(load_specific_raw_data("validation_labels"), load_df(ordner="03_primary", name = "readout_tsfresh_features_validation"))

### Speichern

In [15]:
save_df(df = labels_train_prepare, ordner = "03_primary", name= "labels_train_prepare")
save_df(df = labels_test_prepared, ordner = "03_primary", name= "labels_test_prepared")
save_df(df = labels_validation_prepared, ordner = "03_primary", name= "labels_validation_prepared")

### Feature Selection

#### univariate Cox-Regression

In [53]:
features_train_with_labels = load_df(ordner="03_primary", name = "readout_tsfresh_features_train").merge(load_df(ordner="03_primary", name = "labels_train_prepare")[["duration", "event", "vehicle_id"]], how="left", on="vehicle_id")


In [26]:
display(features_train_with_labels[features_train_with_labels["vehicle_id"]==33335])

,100_0__mean,100_0__median,100_0__standard_deviation,100_0__minimum,100_0__maximum,158_0__mean,158_0__median,158_0__standard_deviation,158_0__minimum,158_0__maximum,158_1__mean,158_1__median,158_1__standard_deviation,158_1__minimum,158_1__maximum,158_2__mean,158_2__median,158_2__standard_deviation,158_2__minimum,158_2__maximum,158_3__mean,158_3__median,158_3__standard_deviation,158_3__minimum,158_3__maximum,158_4__mean,158_4__median,158_4__standard_deviation,158_4__minimum,158_4__maximum,158_5__mean,158_5__median,158_5__standard_deviation,158_5__minimum,158_5__maximum,158_6__mean,158_6__median,158_6__standard_deviation,158_6__minimum,158_6__maximum,158_7__mean,158_7__median,158_7__standard_deviation,158_7__minimum,158_7__maximum,158_8__mean,158_8__median,158_8__standard_deviation,158_8__minimum,158_8__maximum,158_9__mean,158_9__median,158_9__standard_deviation,158_9__minimum,158_9__maximum,167_0__mean,167_0__median,167_0__standard_deviation,167_0__minimum,167_0__maximum,167_1__mean,167_1__median,167_1__standard_deviation,167_1__minimum,167_1__maximum,167_2__mean,167_2__median,167_2__standard_deviation,167_2__minimum,167_2__maximum,167_3__mean,167_3__median,167_3__standard_deviation,167_3__minimum,167_3__maximum,167_4__mean,167_4__median,167_4__standard_deviation,167_4__minimum,167_4__maximum,167_5__mean,167_5__median,167_5__standard_deviation,167_5__minimum,167_5__maximum,167_6__mean,167_6__median,167_6__standard_deviation,167_6__minimum,167_6__maximum,167_7__mean,167_7__median,167_7__standard_deviation,167_7__minimum,167_7__maximum,167_8__mean,167_8__median,167_8__standard_deviation,167_8__minimum,167_8__maximum,167_9__mean,167_9__median,167_9__standard_deviation,167_9__minimum,167_9__maximum,171_0__mean,171_0__median,171_0__standard_deviation,171_0__minimum,171_0__maximum,272_0__mean,272_0__median,272_0__standard_deviation,272_0__minimum,272_0__maximum,272_1__mean,272_1__median,272_1__standard_deviation,272_1__minimum,272_1__maximum,272_2__mean,272_2__median,272_2__standard_deviation,272_2__minimum,272_2__maximum,272_3__mean,272_3__median,272_3__standard_deviation,272_3__minimum,272_3__maximum,272_4__mean,272_4__median,272_4__standard_deviation,272_4__minimum,272_4__maximum,272_5__mean,272_5__median,272_5__standard_deviation,272_5__minimum,272_5__maximum,272_6__mean,272_6__median,272_6__standard_deviation,272_6__minimum,272_6__maximum,272_7__mean,272_7__median,272_7__standard_deviation,272_7__minimum,272_7__maximum,272_8__mean,272_8__median,272_8__standard_deviation,272_8__minimum,272_8__maximum,272_9__mean,272_9__median,272_9__standard_deviation,272_9__minimum,272_9__maximum,291_0__mean,291_0__median,291_0__standard_deviation,291_0__minimum,291_0__maximum,291_1__mean,291_1__median,291_1__standard_deviation,291_1__minimum,291_1__maximum,291_10__mean,291_10__median,291_10__standard_deviation,291_10__minimum,291_10__maximum,291_2__mean,291_2__median,291_2__standard_deviation,291_2__minimum,291_2__maximum,291_3__mean,291_3__median,291_3__standard_deviation,291_3__minimum,291_3__maximum,291_4__mean,291_4__median,291_4__standard_deviation,291_4__minimum,291_4__maximum,291_5__mean,291_5__median,291_5__standard_deviation,291_5__minimum,291_5__maximum,291_6__mean,291_6__median,291_6__standard_deviation,291_6__minimum,291_6__maximum,291_7__mean,291_7__median,291_7__standard_deviation,291_7__minimum,291_7__maximum,291_8__mean,291_8__median,291_8__standard_deviation,291_8__minimum,291_8__maximum,291_9__mean,291_9__median,291_9__standard_deviation,291_9__minimum,291_9__maximum,309_0__mean,309_0__median,309_0__standard_deviation,309_0__minimum,309_0__maximum,370_0__mean,370_0__median,370_0__standard_deviation,370_0__minimum,370_0__maximum,397_0__mean,397_0__median,397_0__standard_deviation,397_0__minimum,397_0__maximum,397_1__mean,397_1__median,397_1__standard_deviation,397_1__minimum,397_1__maximum,397_10__mean,397_10__median,397_10__standard_deviation,397_10__minimum,397_10__maximum,397_11__mean,397_11__median,397_11__standard_deviatio

In [23]:
f_vehicle = features_train_with_labels.sort_values("time_step").groupby("vehicle_id").tail(1)

In [25]:
f_vehicle.head()

,100_0__mean,100_0__median,100_0__standard_deviation,100_0__minimum,100_0__maximum,158_0__mean,158_0__median,158_0__standard_deviation,158_0__minimum,158_0__maximum,158_1__mean,158_1__median,158_1__standard_deviation,158_1__minimum,158_1__maximum,158_2__mean,158_2__median,158_2__standard_deviation,158_2__minimum,158_2__maximum,158_3__mean,158_3__median,158_3__standard_deviation,158_3__minimum,158_3__maximum,158_4__mean,158_4__median,158_4__standard_deviation,158_4__minimum,158_4__maximum,158_5__mean,158_5__median,158_5__standard_deviation,158_5__minimum,158_5__maximum,158_6__mean,158_6__median,158_6__standard_deviation,158_6__minimum,158_6__maximum,158_7__mean,158_7__median,158_7__standard_deviation,158_7__minimum,158_7__maximum,158_8__mean,158_8__median,158_8__standard_deviation,158_8__minimum,158_8__maximum,158_9__mean,158_9__median,158_9__standard_deviation,158_9__minimum,158_9__maximum,167_0__mean,167_0__median,167_0__standard_deviation,167_0__minimum,167_0__maximum,167_1__mean,167_1__median,167_1__standard_deviation,167_1__minimum,167_1__maximum,167_2__mean,167_2__median,167_2__standard_deviation,167_2__minimum,167_2__maximum,167_3__mean,167_3__median,167_3__standard_deviation,167_3__minimum,167_3__maximum,167_4__mean,167_4__median,167_4__standard_deviation,167_4__minimum,167_4__maximum,167_5__mean,167_5__median,167_5__standard_deviation,167_5__minimum,167_5__maximum,167_6__mean,167_6__median,167_6__standard_deviation,167_6__minimum,167_6__maximum,167_7__mean,167_7__median,167_7__standard_deviation,167_7__minimum,167_7__maximum,167_8__mean,167_8__median,167_8__standard_deviation,167_8__minimum,167_8__maximum,167_9__mean,167_9__median,167_9__standard_deviation,167_9__minimum,167_9__maximum,171_0__mean,171_0__median,171_0__standard_deviation,171_0__minimum,171_0__maximum,272_0__mean,272_0__median,272_0__standard_deviation,272_0__minimum,272_0__maximum,272_1__mean,272_1__median,272_1__standard_deviation,272_1__minimum,272_1__maximum,272_2__mean,272_2__median,272_2__standard_deviation,272_2__minimum,272_2__maximum,272_3__mean,272_3__median,272_3__standard_deviation,272_3__minimum,272_3__maximum,272_4__mean,272_4__median,272_4__standard_deviation,272_4__minimum,272_4__maximum,272_5__mean,272_5__median,272_5__standard_deviation,272_5__minimum,272_5__maximum,272_6__mean,272_6__median,272_6__standard_deviation,272_6__minimum,272_6__maximum,272_7__mean,272_7__median,272_7__standard_deviation,272_7__minimum,272_7__maximum,272_8__mean,272_8__median,272_8__standard_deviation,272_8__minimum,272_8__maximum,272_9__mean,272_9__median,272_9__standard_deviation,272_9__minimum,272_9__maximum,291_0__mean,291_0__median,291_0__standard_deviation,291_0__minimum,291_0__maximum,291_1__mean,291_1__median,291_1__standard_deviation,291_1__minimum,291_1__maximum,291_10__mean,291_10__median,291_10__standard_deviation,291_10__minimum,291_10__maximum,291_2__mean,291_2__median,291_2__standard_deviation,291_2__minimum,291_2__maximum,291_3__mean,291_3__median,291_3__standard_deviation,291_3__minimum,291_3__maximum,291_4__mean,291_4__median,291_4__standard_deviation,291_4__minimum,291_4__maximum,291_5__mean,291_5__median,291_5__standard_deviation,291_5__minimum,291_5__maximum,291_6__mean,291_6__median,291_6__standard_deviation,291_6__minimum,291_6__maximum,291_7__mean,291_7__median,291_7__standard_deviation,291_7__minimum,291_7__maximum,291_8__mean,291_8__median,291_8__standard_deviation,291_8__minimum,291_8__maximum,291_9__mean,291_9__median,291_9__standard_deviation,291_9__minimum,291_9__maximum,309_0__mean,309_0__median,309_0__standard_deviation,309_0__minimum,309_0__maximum,370_0__mean,370_0__median,370_0__standard_deviation,370_0__minimum,370_0__maximum,397_0__mean,397_0__median,397_0__standard_deviation,397_0__minimum,397_0__maximum,397_1__mean,397_1__median,397_1__standard_deviation,397_1__minimum,397_1__maximum,397_10__mean,397_10__median,397_10__standard_deviation,397_10__minimum,397_10__maximum,397_11__mean,397_11__median,397_11__standard_deviatio

In [21]:
features_train_with_labels.event.value_counts()

event
0    1016560
1     105801
Name: count, dtype: int64

In [14]:
subset = features_train_with_labels.head()
display(subset)

,100_0__mean,100_0__median,100_0__standard_deviation,100_0__minimum,100_0__maximum,158_0__mean,158_0__median,158_0__standard_deviation,158_0__minimum,158_0__maximum,158_1__mean,158_1__median,158_1__standard_deviation,158_1__minimum,158_1__maximum,158_2__mean,158_2__median,158_2__standard_deviation,158_2__minimum,158_2__maximum,158_3__mean,158_3__median,158_3__standard_deviation,158_3__minimum,158_3__maximum,158_4__mean,158_4__median,158_4__standard_deviation,158_4__minimum,158_4__maximum,158_5__mean,158_5__median,158_5__standard_deviation,158_5__minimum,158_5__maximum,158_6__mean,158_6__median,158_6__standard_deviation,158_6__minimum,158_6__maximum,158_7__mean,158_7__median,158_7__standard_deviation,158_7__minimum,158_7__maximum,158_8__mean,158_8__median,158_8__standard_deviation,158_8__minimum,158_8__maximum,158_9__mean,158_9__median,158_9__standard_deviation,158_9__minimum,158_9__maximum,167_0__mean,167_0__median,167_0__standard_deviation,167_0__minimum,167_0__maximum,167_1__mean,167_1__median,167_1__standard_deviation,167_1__minimum,167_1__maximum,167_2__mean,167_2__median,167_2__standard_deviation,167_2__minimum,167_2__maximum,167_3__mean,167_3__median,167_3__standard_deviation,167_3__minimum,167_3__maximum,167_4__mean,167_4__median,167_4__standard_deviation,167_4__minimum,167_4__maximum,167_5__mean,167_5__median,167_5__standard_deviation,167_5__minimum,167_5__maximum,167_6__mean,167_6__median,167_6__standard_deviation,167_6__minimum,167_6__maximum,167_7__mean,167_7__median,167_7__standard_deviation,167_7__minimum,167_7__maximum,167_8__mean,167_8__median,167_8__standard_deviation,167_8__minimum,167_8__maximum,167_9__mean,167_9__median,167_9__standard_deviation,167_9__minimum,167_9__maximum,171_0__mean,171_0__median,171_0__standard_deviation,171_0__minimum,171_0__maximum,272_0__mean,272_0__median,272_0__standard_deviation,272_0__minimum,272_0__maximum,272_1__mean,272_1__median,272_1__standard_deviation,272_1__minimum,272_1__maximum,272_2__mean,272_2__median,272_2__standard_deviation,272_2__minimum,272_2__maximum,272_3__mean,272_3__median,272_3__standard_deviation,272_3__minimum,272_3__maximum,272_4__mean,272_4__median,272_4__standard_deviation,272_4__minimum,272_4__maximum,272_5__mean,272_5__median,272_5__standard_deviation,272_5__minimum,272_5__maximum,272_6__mean,272_6__median,272_6__standard_deviation,272_6__minimum,272_6__maximum,272_7__mean,272_7__median,272_7__standard_deviation,272_7__minimum,272_7__maximum,272_8__mean,272_8__median,272_8__standard_deviation,272_8__minimum,272_8__maximum,272_9__mean,272_9__median,272_9__standard_deviation,272_9__minimum,272_9__maximum,291_0__mean,291_0__median,291_0__standard_deviation,291_0__minimum,291_0__maximum,291_1__mean,291_1__median,291_1__standard_deviation,291_1__minimum,291_1__maximum,291_10__mean,291_10__median,291_10__standard_deviation,291_10__minimum,291_10__maximum,291_2__mean,291_2__median,291_2__standard_deviation,291_2__minimum,291_2__maximum,291_3__mean,291_3__median,291_3__standard_deviation,291_3__minimum,291_3__maximum,291_4__mean,291_4__median,291_4__standard_deviation,291_4__minimum,291_4__maximum,291_5__mean,291_5__median,291_5__standard_deviation,291_5__minimum,291_5__maximum,291_6__mean,291_6__median,291_6__standard_deviation,291_6__minimum,291_6__maximum,291_7__mean,291_7__median,291_7__standard_deviation,291_7__minimum,291_7__maximum,291_8__mean,291_8__median,291_8__standard_deviation,291_8__minimum,291_8__maximum,291_9__mean,291_9__median,291_9__standard_deviation,291_9__minimum,291_9__maximum,309_0__mean,309_0__median,309_0__standard_deviation,309_0__minimum,309_0__maximum,370_0__mean,370_0__median,370_0__standard_deviation,370_0__minimum,370_0__maximum,397_0__mean,397_0__median,397_0__standard_deviation,397_0__minimum,397_0__maximum,397_1__mean,397_1__median,397_1__standard_deviation,397_1__minimum,397_1__maximum,397_10__mean,397_10__median,397_10__standard_deviation,397_10__minimum,397_10__maximum,397_11__mean,397_11__median,397_11__standard_deviatio

#### Daten für feature selection vorbereiten

In [54]:
def last_readouts(df: pd.DataFrame) -> pd.DataFrame:
    """
    Zieht nur die letzten readouts pro Fahrzeug

    Args:
        df (pd.DataFrame): Original-DataFrame mit 'event'-Spalte

    Returns:
        pd.DataFrame: DF mit nur letzten Readout pro Fahrzeug
    """
    return df.sort_values("time_step").groupby("vehicle_id").tail(1)
    


In [55]:
features_train_with_labels_last = last_readouts(features_train_with_labels)
display(features_train_with_labels_last.shape)


(23548, 529)

In [36]:
del features_train_with_labels
gc.collect() 

5854

#### cox univariate

In [61]:
def cox_univariate_feature_screen(
    df: pd.DataFrame,
    duration_col: str = "duration",
    event_col: str = "event",
) -> Dict[str, Dict[str, float]]:
    """
    Univariate Cox-Regression für alle numerischen Features.
    Features werden global z-standardisiert (HR bezieht sich auf 1 SD).
    Gibt pro Feature Hazard Ratio (HR) und p-Wert zurück.

    Erwartet: numerische Feature-Spalten + 'duration' + 'event'.

    Args:
        df (pd.DataFrame): DataFrame mit den Daten
        duration_col (str): Name der Spalte mit den Überlebenszeiten
        event_col (str): Name der Spalte mit den Ereignisindikatoren
    
    Return:
        Dict[str, Dict[str, float]]: Ergebnisse der univariaten Cox-Regression
    """
    results: Dict[str, Dict[str, float]] = {}

    # Kandidaten: alle numerischen Spalten außer Labels und IDs
    feature_cols = [c for c in df.columns if c not in (duration_col, event_col, "vehicle_id")]

    # Z-Transformation aller Features auf einmal
    df_std = df.copy()
    df_std[feature_cols] = (df_std[feature_cols] - df_std[feature_cols].mean()) / df_std[feature_cols].std(ddof=0)

    cph = CoxPHFitter()

    for col in tqdm(feature_cols, desc="Cox Univariate Feature Screen", unit="feature"):
        try:
            sub = df_std[[duration_col, event_col, col]]
            cph.fit(sub, duration_col=duration_col, event_col=event_col, show_progress=False)
            s = cph.summary.loc[col]
            hr = float(s["exp(coef)"])
            p = float(s["p"])

            results[col] = {"HR": hr, "p": p}
        except Exception:
            continue

    return results


In [78]:
feature_selection_dict = cox_univariate_feature_screen(features_train_with_labels_last)

Cox Univariate Feature Screen:   1%|▏         | 7/526 [00:02<02:46,  3.11feature/s]


KeyboardInterrupt: 

In [79]:
display(feature_selection_dict)

{'100_0__mean': {'HR': 0.9750382815136859, 'p': 0.28899116189267426},
 '100_0__median': {'HR': 0.9760481612698878, 'p': 0.3074903170152},
 '100_0__standard_deviation': {'HR': 1.0085275539003582,
  'p': 0.6639944645875869},
 '100_0__minimum': {'HR': 0.9705392480559696, 'p': 0.21450608710252897},
 '100_0__maximum': {'HR': 0.9821893216045807, 'p': 0.43241068720751774},
 '158_0__mean': {'HR': 1.0444812062157105, 'p': 0.022583230278183524},
 '158_0__median': {'HR': 1.0465640533266276, 'p': 0.01589316624036952},
 '158_0__standard_deviation': {'HR': 1.0338362850218212,
  'p': 0.045670309786400666},
 '158_0__minimum': {'HR': 1.0337809386267176, 'p': 0.09511505190640347},
 '158_0__maximum': {'HR': 1.0503416663303606, 'p': 0.008751687501202832},
 '158_1__mean': {'HR': 1.0332711275426245, 'p': 0.14113098199178264},
 '158_1__median': {'HR': 1.031815810294277, 'p': 0.1599289156866742},
 '158_1__standard_deviation': {'HR': 1.055379500907314,
  'p': 0.0029437120956486438},
 '158_1__minimum': {'HR': 1

In [89]:
# Speichern
with open("../data/03_primary/feature_selection_dict.json", "w") as f:
    json.dump(feature_selection_dict, f, indent=2)

# Laden
with open("../data/03_primary/feature_selection_dict.json", "r") as f:
    feature_selection_dict = json.load(f)


In [91]:
cox_p_werte = pd.DataFrame(feature_selection_dict).T.sort_values("p").head(10)

In [92]:
display(cox_p_werte)

,HR,p
time_step,1.060433e-01,0.000000e+00
397_29__maximum,1.131582e-06,3.137288e-69
397_17__standard_deviation,9.883996e-10,1.091842e-57
397_33__maximum,1.149328e+00,1.645207e-33
397_27__maximum,1.102657e+00,6.099848e-32
158_9__maximum,1.150421e+00,4.014737e-29
459_9__maximum,1.173067e+00,6.048633e-29
397_27__mean,1.122139e+00,6.088735e-26
397_34__maximum,1.093917e+00,5.638176e-25
397_27__median,1.120108e+00,8.166440e-25


In [93]:
def auswahl_features_p(feature_selection_dict: dict, p_threshold: float) -> dict:
    """
    Select features based on p-value threshold.

    Args:
        feature_selection_dict (dict): Dictionary containing feature names as keys and their p-values as values.
        p_threshold (float): The p-value threshold for feature selection.

    Returns:
        dict: A dictionary containing the selected features.
    """
    
    selected_features = {}
    for key, value in feature_selection_dict.items():
        if value['p'] <= p_threshold:
            selected_features[key] = value
    return selected_features

In [94]:
selected_features_p005 = auswahl_features_p(feature_selection_dict, p_threshold=0.01)

print(f"Original feature count: {len(feature_selection_dict)}, Selected feature count: {len(selected_features_p005)}")

Original feature count: 526, Selected feature count: 292


### Pearsonkorrelationsmatrix

In [45]:
features_train = load_df(ordner="03_primary", name = "readout_tsfresh_features_train")


In [44]:
def drop_correlated_features(results_dict: dict, df: pd.DataFrame, threshold: float) -> dict:
    """
    Entfernt korrelierte Features basierend auf Pearson-Korrelation.
    Behalte jeweils das Feature mit kleinerem p-Wert aus results_dict.
    
    Args:
        results_dict (dict): {feature: {"HR":..., "p":..., "p_ph":...}}
        df (pd.DataFrame): Standardisierte Feature-Matrix (numerisch).
        threshold (float): Schwellwert für Korrelation.
    
    Returns:
        dict: Gefilterte results_dict ohne stark korrelierte Features.
    """
    features = list(results_dict.keys())
    corr = df[features].corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    
    to_drop = set()
    for col in upper.columns:
        for row in upper.index:
            if row in to_drop or col in to_drop:
                continue
            if upper.loc[row, col] > threshold:
                # Wähle das Feature mit kleinerem p-Wert
                p_row = results_dict[row]["p"]
                p_col = results_dict[col]["p"]
                if p_row <= p_col:
                    to_drop.add(col)
                else:
                    to_drop.add(row)
    
    return {f: results_dict[f] for f in results_dict if f not in to_drop}


In [100]:
selected_features_p005_corr07 = drop_correlated_features(selected_features_p005, features_train, threshold=0.7)
selected_features_p005_corr05 = drop_correlated_features(selected_features_p005, features_train, threshold=0.5)
print(f"Original feature count: {len(feature_selection_dict)}, Selected feature count: {len(selected_features_p005)}, Selected feature count after correlation removal with 0.7: {len(selected_features_p005_corr07)}, 0.5: {len(selected_features_p005_corr05)}")

Original feature count: 526, Selected feature count: 292, Selected feature count after correlation removal with 0.7: 36, 0.5: 14


In [101]:
display(selected_features_p005_corr05)

{'272_2__mean': {'HR': 0.790807902794873, 'p': 1.4978658945443914e-10},
 '272_5__standard_deviation': {'HR': 1.0271457254767,
  'p': 0.009014588588221965},
 '291_10__maximum': {'HR': 1.0858754997904623, 'p': 1.0666144822635168e-05},
 '397_23__standard_deviation': {'HR': 0.09864183045322213,
  'p': 1.5441498666431777e-24},
 '397_29__maximum': {'HR': 1.1315824359713941e-06,
  'p': 3.1372877174957393e-69},
 '397_33__maximum': {'HR': 1.1493284976692406, 'p': 1.6452067304154674e-33},
 '397_34__standard_deviation': {'HR': 1.1123908413910812,
  'p': 2.473566179941572e-24},
 '397_35__maximum': {'HR': 1.0513485812417007, 'p': 1.0630781819076172e-12},
 '397_5__standard_deviation': {'HR': 1.0434205838063615,
  'p': 0.0014657832536113895},
 '459_15__maximum': {'HR': 1.1262774844705816, 'p': 6.811258033773739e-12},
 '459_5__maximum': {'HR': 1.094285046273881, 'p': 1.5101572602507294e-10},
 '459_9__standard_deviation': {'HR': 1.1000935512205194,
  'p': 2.8733077265537955e-17},
 '837_0__maximum': {'H

### relevante Featues selektieren

In [110]:
def select_relevant_features(df_readouts: pd.DataFrame, selected_features: dict, df_labels: pd.DataFrame) -> pd.DataFrame:
    """
    Reduziert DataFrame auf relevante Features (nach Feature Selection) + Labels.
    
    Args:
        df (pd.DataFrame): Ursprüngliches DataFrame mit allen Features und Labels.
        selected_features (dict): Dict der Form {feature: {"HR":..., "p":..., "p_ph":...}}
                                  aus der Feature Selection.
        df_labels (pd.DataFrame): DataFrame mit den Labelspalten.

    Returns:
        pd.DataFrame: DataFrame mit ausgewählten Features und Labelspalten.
    """
    # Relevante Spalten aus dem Dict
    feature_cols = list(selected_features.keys())
    feature_cols.append("vehicle_id")

    df_readouts = df_readouts[feature_cols]

    df_readouts = df_readouts.merge(df_labels, how="left", on="vehicle_id")

    return df_readouts


In [114]:
feature_train_corr05 = select_relevant_features(load_df(ordner="03_primary", name = "readout_tsfresh_features_train"), selected_features_p005_corr05, load_df(ordner="03_primary", name = "labels_train_prepare"))
feature_train_corr07 = select_relevant_features(load_df(ordner="03_primary", name = "readout_tsfresh_features_train"), selected_features_p005_corr07, load_df(ordner="03_primary", name = "labels_train_prepare"))

feature_test_corr05 = select_relevant_features(load_df(ordner="03_primary", name = "readout_tsfresh_features_test"), selected_features_p005_corr05, load_df(ordner="03_primary", name = "labels_test_prepared"))
feature_test_corr07 = select_relevant_features(load_df(ordner="03_primary", name = "readout_tsfresh_features_test"), selected_features_p005_corr07, load_df(ordner="03_primary", name = "labels_test_prepared"))

feature_validation_corr05 = select_relevant_features(load_df(ordner="03_primary", name = "readout_tsfresh_features_validation"), selected_features_p005_corr05, load_df(ordner="03_primary", name = "labels_validation_prepared"))
feature_validation_corr07 = select_relevant_features(load_df(ordner="03_primary", name = "readout_tsfresh_features_validation"), selected_features_p005_corr07, load_df(ordner="03_primary", name = "labels_validation_prepared"))

In [116]:
save_df(df = feature_train_corr05, ordner="04_feature", name = "feature_train_corr05")
save_df(df = feature_train_corr07, ordner="04_feature", name = "feature_train_corr07")

save_df(df = feature_test_corr05, ordner="04_feature", name = "feature_test_corr05")
save_df(df = feature_test_corr07, ordner="04_feature", name = "feature_test_corr07")

save_df(df = feature_validation_corr05, ordner="04_feature", name = "feature_validation_corr05")
save_df(df = feature_validation_corr07, ordner="04_feature", name = "feature_validation_corr07")
